Department of Physics, University of Pisa AA 2019-2020)

### Multimessenger Physics Laboratory tutorial series



## Tutorial 1 - Reading CSV files
#### (M. Razzano, Mar 11, 2020)

In this tutorial we will learn how to use Python to open Comma Separated Value (CSV) files. These CSV files are ASCII files that contains sets of data organized in columns, each one divided by a special separator character, which could be a comma or something else (like a space, or a tab).
CSV can be read efficiently using Pandas, but in some cases it is useful to learn to read and visualize files, in order to manage more complicate cases, where headers are put at the beginning of the file and you need to read the information stored in it

### Basic Setup
First of all, we import the required modules and we download the data file.

In [25]:
import os   #These two are useful modules
import sys

import numpy as np #Numerical Python

**Important** Within the notebook you can run BASH command, by putting a ! before the command. For instance, if you want to run a ls command, type !ls

In [26]:
# you can run the pwd command (Print Working Directory) to see your directory
!pwd

/home/jovyan/work/grlmrc/PLS/Code


In [27]:
#or you can run ls command to list the directory where you are
# (-l indicates that you can see all the properties of the file)
!ls -l

total 1344
-rw-r--r-- 1 jovyan users  225118 Feb 16 16:39 ingegneria_cisia.ipynb
-rw-r--r-- 1 jovyan users 1131630 Feb 16 16:43 scienze_cisia.ipynb
-rw-r--r-- 1 jovyan users   16370 Feb 16 16:09 scienze_cisia_read.ipynb


In [28]:
#or the one above you
!ls -l ../

total 12
drwxr-sr-x 3 jovyan users 4096 Feb 16 16:43 Code
drwxr-sr-x 3 jovyan users 4096 Feb 16 16:38 Data
drwxr-sr-x 2 jovyan users 4096 Feb 16 14:02 results


### Le't retrieve the data
The data are stored in a "data" directory at the same level of the "code" directory where you are now. Let's do some work to navigate the directory...
We will use the *os* module, in particular the *os.path* submodule.

**Note**: In this case we define the directories directly in this notebook, instead of using environment variables. For this examples, it is easier...

In [29]:
# Let's define te directory where you are with the "get current working directory" command...
work_dir = os.getcwd()
print("I am in dir %s" % work_dir)

#We can go up by using the path.dirname
main_dir =os.path.dirname(work_dir)
print("Main directory is %s" % main_dir)

#We can then define the data directory using the os.path.join command
data_dir = os.path.join(main_dir,"Data")
print("data dir is %s" % data_dir)

#..and we can define some output directory with the results
results_dir = os.path.join(main_dir,"results")

#if the directory does not exist, you can create it...
if not os.path.exists(results_dir):
    os.mkdir(results_dir)
    print("Directory %s created !" % results_dir)

I am in dir /home/jovyan/work/grlmrc/PLS/Code
Main directory is /home/jovyan/work/grlmrc/PLS
data dir is /home/jovyan/work/grlmrc/PLS/Data


### Opening and exploring a file
First of all, we will open a file and explore it, by excluding comments, and deciding how many data lines exists

In [30]:
#define the filename, in this case is in the hw-data/hw01-data directory
data_filename = "CISIA_AI04_Scienze.txt"

#we use the open command
# IMPORTANT! The file is in the data directory, so we need to use the join command

data_file = open(os.path.join(data_dir,data_filename),"r")  #r means reading mode

#the readlines function gives a list with a line each
lines = data_file.readlines()

#let's print the first 20 lines...
#print(lines)

In [31]:
# We can ue the split function to split a long string. 
# Here we use the comma as separator, but one can use also other characters
studenti = lines[0].split('{"codiceStudente":')
studenti_eff=[]
cod_fis=[]
#print("file contains %d students: \n" % (len(studenti)-1))
#print(len(studenti[2]))

for li in range(1,len(studenti)):
    if len(studenti[li])>100:
        studenti_eff.append(studenti[li])

print("file contains %d students, but only %d have datas associated: \n" % ((len(studenti)-1),len(studenti_eff)))
#print("file contains %d students: \n" % len(studenti_eff))
for li in range(0,len(studenti_eff)):
    print(studenti_eff[li],'\n')
    #print(studenti_eff[li][1:17])
    cod_fis.append(studenti_eff[li][1:17])
    


file contains 1923 students, but only 391 have datas associated: 

"CPNTSM00D69I330H","elencoRisultatiTest":[{"data":"2019-09-12","sede":"Universita' di Pisa, sede di PISA SCIENZE MATEMATICA","punteggio":"39.5","risposteCorrette":"41","risposteErrate":"10","risposteNulle":"29","elencoRisulatiSezione":[{"codice":"TCSC01","punteggio":"9.25","risposteCorrette":"10","risposteErrate":"3","risposteNulle":"7"},{"codice":"TCSC02","punteggio":"1.5","risposteCorrette":"2","risposteErrate":"2","risposteNulle":"6"},{"codice":"TCSC03","punteggio":"7","risposteCorrette":"7","risposteErrate":"0","risposteNulle":"3"},{"codice":"TCSC04","punteggio":"3.75","risposteCorrette":"4","risposteErrate":"1","risposteNulle":"5"},{"codice":"TCSC05","punteggio":"18","risposteCorrette":"18","risposteErrate":"4","risposteNulle":"8"}]},{"data":"2019-10-18","sede":"Universita' di Pisa, sede di PISA SCIENZE MATEMATICA","punteggio":"32.25","risposteCorrette":"33","risposteErrate":"5","risposteNulle":"42","elencoRisulati

In [32]:
import re
dictio={}
for per in range(len(cod_fis)):
    studente_temp = studenti_eff[per].split('{"data":')
    dictio[cod_fis[per]]={}
    for sez in range(1,len(studente_temp)):
        dictio[cod_fis[per]][sez]={}
        dati_studente_codici = studente_temp[sez].split('{"codice":"')
        dati_studente_sez = studente_temp[sez].split('","')
        data=dati_studente_sez[0][len('"'):]
        sede=dati_studente_sez[1][len('sede":"'):]
        punteggio=np.float(np.asarray(re.findall(r"[-+]?\d*\.\d+|\d+", dati_studente_sez[2])))
        ris_corr=np.float(np.asarray(re.findall(r"[-+]?\d*\.\d+|\d+", dati_studente_sez[3])))
        ris_err=np.float(np.asarray(re.findall(r"[-+]?\d*\.\d+|\d+", dati_studente_sez[4])))
        ris_null=np.float(np.asarray(re.findall(r"[-+]?\d*\.\d+|\d+", dati_studente_sez[5])))
        for cod in range(1,len(dati_studente_codici)):
            altro=dati_studente_codici[cod].split('","')
            code=altro[0]
            punt_code=np.float(np.asarray(re.findall(r"[-+]?\d*\.\d+|\d+", altro[1])))
            ris_corr_code=np.float(np.asarray(re.findall(r"[-+]?\d*\.\d+|\d+", altro[2])))
            ris_err_code=np.float(np.asarray(re.findall(r"[-+]?\d*\.\d+|\d+", altro[3])))
            ris_null_code=np.float(np.asarray(re.findall(r"[-+]?\d*\.\d+|\d+", altro[4])))
            
            dictio[cod_fis[per]][sez][cod]=[data,sede,punteggio,ris_corr,ris_err,ris_null,code,punt_code,ris_corr_code,ris_err_code,ris_null_code]
        

In [22]:
count=0
for a in range(len(cod_fis)):
    print('\n')
    print(cod_fis[a])
    for b in range(1,5):
        if b in dictio[cod_fis[a]].keys():
            print('Tentativo numero %d'%b)
            for c in range(1,15):
                if c in dictio[cod_fis[a]][b].keys():
                    print(dictio[cod_fis[a]][b][c])
                    count=count+1
print(count)



CPNTSM00D69I330H
Tentativo numero 1
['2019-09-12', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 39.5, 41.0, 10.0]
['2019-09-12', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 39.5, 41.0, 10.0, 29.0, 'TCSC02', 1.5, 2.0, 2.0, 6.0]
['2019-09-12', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 39.5, 41.0, 10.0, 29.0, 'TCSC03', 7.0, 7.0, 0.0, 3.0]
['2019-09-12', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 39.5, 41.0, 10.0, 29.0, 'TCSC04', 3.75, 4.0, 1.0, 5.0]
['2019-09-12', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 39.5, 41.0, 10.0, 29.0, 'TCSC05', 18.0, 18.0, 4.0, 8.0]
Tentativo numero 2
['2019-10-18', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 32.25, 33.0, 5.0]
['2019-10-18', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 32.25, 33.0, 5.0, 42.0, 'TCSC02', 4.0, 4.0, 0.0, 6.0]
['2019-10-18', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 32.25, 33.0, 5.0, 42.0, 'TCSC03', 3.75, 4.0, 1.0, 5.0]
['2019-10-1

In [33]:
count=0
file_uno = open(data_dir + "/dati_ordinati_ridotti_CISIA_AI04_Scienze.txt", "w")
file_uno.write("Codice fiscale; Tentativo; Data; Sede; Punteggio; Risposte corrette; Risposte errate; Risposte Nulle" + '\n')
for a in range(len(cod_fis)):
    print('\n')
    print(cod_fis[a])
    #file_uno.write(cod_fis[a]+', ')
    for b in range(1,5):
        if b in dictio[cod_fis[a]].keys():
            print('Tentativo numero %d'%b)
            file_uno.write(cod_fis[a]+'; ')
            file_uno.write('Tentativo numero %d; '%b)
            st=dictio[cod_fis[a]][b][1] 
            for i in range(0, len(st) - 6 ): st.pop() 
            listToStr1 = '; '.join([str(elem) for elem in st])
            file_uno.write(listToStr1 + '\n')
            for c in range(1,15):
                if c in dictio[cod_fis[a]][b].keys():
                    s=dictio[cod_fis[a]][b][c]

                    print(s)
                    #print(dictio[cod_fis[a]][b][1])
                    #listToStr = ' '.join([str(elem) for elem in s])
                    #print(listToStr)
                    #file_uno.write(listToStr) 
                    #count=count+1
file_uno.close()
#print(count)
#print(dictio[cod_fis[0]][1][2])
#print(dictio[cod_fis[a]][b].keys())



CPNTSM00D69I330H
Tentativo numero 1
['2019-09-12', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 39.5, 41.0, 10.0, 29.0]
['2019-09-12', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 39.5, 41.0, 10.0, 29.0, 'TCSC02', 1.5, 2.0, 2.0, 6.0]
['2019-09-12', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 39.5, 41.0, 10.0, 29.0, 'TCSC03', 7.0, 7.0, 0.0, 3.0]
['2019-09-12', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 39.5, 41.0, 10.0, 29.0, 'TCSC04', 3.75, 4.0, 1.0, 5.0]
['2019-09-12', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 39.5, 41.0, 10.0, 29.0, 'TCSC05', 18.0, 18.0, 4.0, 8.0]
Tentativo numero 2
['2019-10-18', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 32.25, 33.0, 5.0, 42.0]
['2019-10-18', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 32.25, 33.0, 5.0, 42.0, 'TCSC02', 4.0, 4.0, 0.0, 6.0]
['2019-10-18', "Universita' di Pisa, sede di PISA SCIENZE MATEMATICA", 32.25, 33.0, 5.0, 42.0, 'TCSC03', 3.75, 4.0, 1.0, 5.0]